In [99]:
import argparse
import datetime
import time
import cv2
import numpy as np
import os
import math
import yaml

In [2]:
path = "/home/joe/Videos/Webcam/"
fileName = "2022-05-04-214101.webm"
filePath = path + fileName
camera = cv2.VideoCapture(filePath)

In [3]:
pinkLower = (140, 10, 225)
pinkUpper = (180, 118, 255)

redLower = (17, 15, 100)
redUpper = (50, 56, 200)

upper = pinkUpper
lower = redUpper

In [4]:
(grabbed, frame) = camera.read()
height, width = frame.shape[:2]
w = width
h = height



In [5]:
"""
calibrationFile = open('calibrations/calibration2.yaml', 'r')
calibrationData = yaml.safe_load(calibrationFile)

camera_matrix = calibrationData['camera_matrix']
numColumns = camera_matrix['cols']
numRows =  camera_matrix['rows']
cameraMatrix = np.array(camera_matrix['data'])
cameraMatrix = np.reshape(cameraMatrix, (numRows, numColumns))

distortion_coefficients = calibrationData['distortion_coefficients']
numColumns = distortion_coefficients['cols']
numRows =  distortion_coefficients['rows']
distortionCoefficients = np.array(distortion_coefficients['data'])
#distortionCoefficients = np.reshape(distortionCoefficients, (numRows, numColumns))
"""
mtx = [[798.86256178, 0, 401.52277111], [ 0, 812.90563717,319.32828483], [  0, 0,1.]]
cameraMatrix = np.reshape(mtx, (3, 3))
dist = [-3.51591693e-01, 1.92604733e-01, 3.20674878e-04, 1.56190371e-04, -1.16111572e-01]

distortionCoefficients = np.array(dist)



In [33]:
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(cameraMatrix,distortionCoefficients,(w,h),1,(w,h))
frame = cv2.undistort(frame, cameraMatrix, distortionCoefficients, None, newcameramtx) 


In [34]:
diagnosticFrame = frame.copy()

In [35]:
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
arucoParams = cv2.aruco.DetectorParameters_create()


ballDetectionCorners = [(0,0), (0, height), (height,width), (0,width)]

In [114]:
(arucoCorners, ids, rejected) = cv2.aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

In [37]:
if(len(arucoCorners) > 0):
		ids = ids.flatten()

		for(markerCorner, markerID) in zip(arucoCorners, ids):
			corners = markerCorner.reshape((4,2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners


			# convert each of the (x, y)-coordinate pairs to integers
			topRight = (int(topRight[0]), int(topRight[1]))
			bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
			bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
			topLeft = (int(topLeft[0]), int(topLeft[1]))


			cv2.line(diagnosticFrame, topLeft, topRight, (0, 255, 0), 2)
			cv2.line(diagnosticFrame, topRight, bottomRight, (0, 255, 0), 2)
			cv2.line(diagnosticFrame, bottomRight, bottomLeft, (0, 255, 0), 2)
			cv2.line(diagnosticFrame, bottomLeft, topLeft, (0, 255, 0), 2)

			if(markerID >= 0 and markerID <= 3):
				newCorner = (0,0)
				if(markerID == 0):
					newCorner = topRight
				elif (markerID == 1):
					newCorner = bottomRight
				elif (markerID == 2):
					newCorner = bottomLeft
				elif (markerID == 3):
					newCorner = topLeft
				ballDetectionCorners[markerID] = newCorner

In [38]:
#draw bounding box
boundingBoxPoints = np.array(ballDetectionCorners)
boundingBoxPoints.reshape((-1,1,2))
cv2.polylines(diagnosticFrame, [boundingBoxPoints], True, (255, 0, 0), 2)

array([[[188, 188, 188],
        [181, 181, 181],
        [145, 145, 145],
        ...,
        [  0,   0,   0],
        [  7,   7,   8],
        [  6,   6,   7]],

       [[187, 187, 187],
        [186, 186, 186],
        [186, 186, 186],
        ...,
        [ 63,  67,  71],
        [ 64,  69,  74],
        [ 12,  12,  13]],

       [[156, 156, 156],
        [186, 186, 186],
        [186, 186, 186],
        ...,
        [ 79,  84,  90],
        [ 63,  67,  72],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [39]:
markerLength = 0.02 #2cm, 0.02m
distCoeffs = distortionCoefficients
(rvecs, tvecs, points) = cv2.aruco.estimatePoseSingleMarkers(arucoCorners, markerLength, cameraMatrix, distCoeffs)

In [40]:
for rvec, tvec in zip(rvecs, tvecs):
    diagnosticFrame = cv2.drawFrameAxes(diagnosticFrame, cameraMatrix, distCoeffs, rvec, tvec, 0.05)

In [41]:
cv2.imwrite("test_img9.png", diagnosticFrame)

True

In [18]:
camera.release()

In [42]:
print(rvecs)

[[[-0.13289384  2.77569878  0.38300636]]

 [[-0.02109468 -2.90278159 -0.1539173 ]]

 [[ 0.12421692 -3.06831899 -0.9561763 ]]

 [[-0.05134248 -3.09628212 -0.64108994]]]


In [28]:
board = cv2.aruco.Board_create(arucoCorners, arucoDict, ids)


error: OpenCV(4.5.5) /io/opencv_contrib/modules/aruco/src/aruco.cpp:1516: error: (-215:Assertion failed) objPoints.type() == CV_32FC3 || objPoints.type() == CV_32FC1 in function 'create'


In [27]:

(retval, rvec, tvec) =  cv2.aruco.estimatePoseSingleMarkers(arucoCorners, markerLength, cameraMatrix, distCoeffs)

In [23]:
from inspect import getmembers, isfunction
print(getmembers(cv2.aruco, isfunction))

[]


In [43]:
print(arucoCorners)
print(ids)

(array([[[ 96., 476.],
        [ 70., 475.],
        [ 69., 450.],
        [ 96., 451.]]], dtype=float32), array([[[662., 463.],
        [638., 463.],
        [639., 439.],
        [663., 439.]]], dtype=float32), array([[[ 70., 106.],
        [ 42., 106.],
        [ 40.,  77.],
        [ 68.,  76.]]], dtype=float32), array([[[674., 103.],
        [648., 102.],
        [649.,  74.],
        [675.,  75.]]], dtype=float32))
[3 0 2 1]


In [70]:
arucoMarker = arucoCorners[3].reshape((4,2))

In [71]:
print(arucoMarker)

[[674. 103.]
 [648. 102.]
 [649.  74.]
 [675.  75.]]


In [72]:
topRight = arucoMarker[0]
topLeft = arucoMarker[1]
bottomLeft = arucoMarker[2]
bottomRight = arucoMarker[3]

pixelWidth = topRight[0] - topLeft[0]
pixelHeight = topRight[1] - bottomRight[1]

In [73]:
print(pixelWidth)
print(pixelHeight)

26.0
28.0


In [74]:
fx = cameraMatrix[0][0]
fy = cameraMatrix[1][1]

cx = cameraMatrix[0][2]
cy = cameraMatrix[1][2]

In [75]:
print(fx,fy,cx,cy)
print(cameraMatrix)

798.86256178 812.90563717 401.52277111 319.32828483
[[798.86256178   0.         401.52277111]
 [  0.         812.90563717 319.32828483]
 [  0.           0.           1.        ]]


In [76]:
knownLen = 0.02 #m
W_obj = knownLen
distance_from_dx = fx * W_obj / pixelWidth * 100 #cm

H_obj = knownLen
distance_from_dy = fy * H_obj / pixelHeight * 100 #cm


In [77]:
print("distance from dx (cm)", distance_from_dx)
print("distance from dy (cm)", distance_from_dy)

distance from dx (cm) 61.45096629076923
distance from dy (cm) 58.06468836928571


In [79]:
#manual estimation of focal length

knownDistance = 0.46 #m
Z_obj = knownDistance

fx_est = pixelWidth * Z_obj / W_obj

fy_est = pixelHeight * Z_obj / H_obj



In [80]:
print("fx_est", fx_est)
print("fy_est", fy_est)

fx_est 598.0
fy_est 644.0


In [86]:
estimatedFocalLengths = []
estimatedDistances = []

for markerCorner in arucoCorners:
    corners = markerCorner.reshape((4,2))
    (topRight, topLeft, bottomLeft, bottomRight) = corners

    pixelWidth = topRight[0] - topLeft[0]
    pixelHeight = topRight[1] - bottomRight[1]

    fx_est = pixelWidth * Z_obj / W_obj
    fy_est = pixelHeight * Z_obj / H_obj
    estimatedFocalLengths.append((fx_est, fy_est))
    
    distance_from_dx = fx * W_obj / pixelWidth * 100 #cm
    distance_from_dy = fy * H_obj / pixelHeight * 100 #cm
    
    estimatedDistances.append((distance_from_dx, distance_from_dy))

In [87]:
print(estimatedFocalLengths)

[(598.0, 575.0), (552.0, 552.0), (644.0, 690.0), (598.0, 644.0)]


In [88]:
print(estimatedDistances)

[(61.45096629076923, 65.03245097359999), (66.57188014833334, 67.74213643083333), (57.061611555714286, 54.193709144666656), (61.45096629076923, 58.06468836928571)]


In [95]:
#height between top of marker 0 and bottom marker 1

H_m0_m1 = 0.28 #0.28 m or 28 cm
distance = knownDistance

marker0 = arucoCorners[1].reshape((4,2))
(m0_topLeft, m0_topRight, m0_bottomRight, m0_bottomleft) = marker0

marker1 = arucoCorners[3].reshape((4,2))
(m1_topLeft, m1_topRight, m1_bottomRight, m1_bottomleft) = marker1

pixelHeight = m0_bottomLeft[1] - m1_topLeft[1]

apparentHeight_0 = distance * pixelHeight /  fy
apparentHeight_1 = distance * pixelHeight /  estimatedFocalLengths[0][1]

print(apparentHeight_0, apparentHeight_1)

0.1901327693311068 0.2688


In [97]:
z0 = estimatedDistances[1][0]
z1 = estimatedDistances[3][0]

delta_z = abs(z0 - z1)

print(z0, z1, delta_z)

66.57188014833334 61.45096629076923 5.120913857564105


In [105]:
theta0 = math.atan2(apparentHeight_0, delta_z) * 180/math.pi

theta1 = math.atan2(apparentHeight_1, delta_z) * 180/math.pi

print(theta0, theta1)

2.1263399120756765 3.0047340343371736


In [104]:
h = H_m0_m1
a0 = apparentHeight_0

theta2 = math.acos(a0/h) * 180/math.pi

a1 = apparentHeight_1
theta3 = math.acos(a1/h) * 180/math.pi

print(theta2, theta3)

47.23089182494468 16.26020470831199


In [177]:

bl = np.int32(arucoCorners[0].reshape((4,2))[1])
bl = tuple(bl)
                             
br = np.int32(arucoCorners[1].reshape((4,2))[0])
br = tuple(br)

tl = np.int32(arucoCorners[2].reshape((4,2))[2])
tl = tuple(tl)

tr = np.int32(arucoCorners[3].reshape((4,2))[3])
tr = tuple(tr)

pointsBefore = [tl, tr, br, bl]

print("tl, tr, br, bl")
print(pointsBefore)
print(ids)

tl, tr, br, bl
[(40, 77), (675, 75), (662, 463), (70, 475)]
[[3]
 [0]
 [2]
 [1]]


In [178]:
pixelWidth1 = np.abs(tl[0] - tr[0])
pixelWidth2 = np.abs(bl[0] - br[0])


pixelHeight1 = np.abs(tl[1] - bl[1])
pixelHeight2 = np.abs(br[1] - tr[1])

print(pixelWidth1, pixelWidth2, pixelHeight1, pixelHeight2)

635 592 398 388


In [179]:
newWidth = max(pixelWidth1, pixelWidth2)
newHeight = max(pixelHeight1, pixelHeight2)

In [180]:
#tl, tr, br, bl
newCoordinates = [(0,0), (newWidth, 0), (newWidth, newHeight), (0, newHeight)]
print(newCoordinates)

pointsBefore = np.float32(pointsBefore)
newCoordinates = np.float32(newCoordinates)

[(0, 0), (635, 0), (635, 398), (0, 398)]


In [181]:
M = cv2.getPerspectiveTransform(pointsBefore,newCoordinates)

In [182]:
boardImg = cv2.warpPerspective(newFrame, M, (int(newWidth), int(newHeight)))
cv2.imwrite("boardImg.png", boardImg)


radius = 5
newFrame = frame.copy()
cv2.circle(newFrame, (int(tl[0]), int(tl[1])), int(radius), (0, 255, 255), 2)
cv2.circle(newFrame, (int(bl[0]), int(bl[1])), int(radius), (0, 255, 255), 2)
cv2.circle(newFrame, (int(tr[0]), int(tr[1])), int(radius), (0, 255, 255), 2)
cv2.circle(newFrame, (int(br[0]), int(br[1])), int(radius), (0, 255, 255), 2)
cv2.imwrite("diagnosticFrame.png", newFrame)

True

In [161]:
print((newWidth, newHeight))

(543.0, 345.0)


In [162]:
print(np.shape(boardImg))

(543, 345, 3)


In [159]:
print(np.shape(diagnosticFrame))

(600, 800, 3)
